In [1]:
#######################################################################
# Copyright (C)                                                       #
# 2016 Shangtong Zhang(zhangshangtong.cpp@gmail.com)                  #
# 2016 Kenta Shimada(hyperkentakun@gmail.com)                         #
# 2017 Ji Yang(jyang7@ualberta.ca)                                    #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

"""
Simulation of applying the in-place version of iterative policy evaluation algorithm in
the grid world, from Sutton & Barto's RL Book, Section 4.1 Policy Evaluation (Prediction)
"""

from __future__ import print_function
import numpy as np

WORLD_SIZE = 4
REWARD = -1.0  # a constant reward for moving to non-terminal states
ACTION_PROB = 0.25  # random policy
GAMMA = 1.0  # episode task, no discount

world = np.zeros((WORLD_SIZE, WORLD_SIZE))

# left, up, right, down
actions = ['L', 'U', 'R', 'D']

In [2]:
# here we generate the possible next state for each cell in the grid world
next_state = []
for i in range(0, WORLD_SIZE):
    next_state.append([])
    for j in range(0, WORLD_SIZE):
        next = dict()
        # uncomment this print and the last print in this loop if you haven't got it
        print('Grid cell (col, row):', i, j, end=' ')

        if i == 0:
            next['U'] = [i, j]
        else:
            next['U'] = [i - 1, j]

        if i == WORLD_SIZE - 1:
            next['D'] = [i, j]
        else:
            next['D'] = [i + 1, j]

        if j == 0:
            next['L'] = [i, j]
        else:
            next['L'] = [i, j - 1]

        if j == WORLD_SIZE - 1:
            next['R'] = [i, j]
        else:
            next['R'] = [i, j + 1]

        print(next)

        next_state[i].append(next)

Grid cell (col, row): 0 0 {'R': [0, 1], 'D': [1, 0], 'L': [0, 0], 'U': [0, 0]}
Grid cell (col, row): 0 1 {'R': [0, 2], 'D': [1, 1], 'L': [0, 0], 'U': [0, 1]}
Grid cell (col, row): 0 2 {'R': [0, 3], 'D': [1, 2], 'L': [0, 1], 'U': [0, 2]}
Grid cell (col, row): 0 3 {'R': [0, 3], 'D': [1, 3], 'L': [0, 2], 'U': [0, 3]}
Grid cell (col, row): 1 0 {'R': [1, 1], 'D': [2, 0], 'L': [1, 0], 'U': [0, 0]}
Grid cell (col, row): 1 1 {'R': [1, 2], 'D': [2, 1], 'L': [1, 0], 'U': [0, 1]}
Grid cell (col, row): 1 2 {'R': [1, 3], 'D': [2, 2], 'L': [1, 1], 'U': [0, 2]}
Grid cell (col, row): 1 3 {'R': [1, 3], 'D': [2, 3], 'L': [1, 2], 'U': [0, 3]}
Grid cell (col, row): 2 0 {'R': [2, 1], 'D': [3, 0], 'L': [2, 0], 'U': [1, 0]}
Grid cell (col, row): 2 1 {'R': [2, 2], 'D': [3, 1], 'L': [2, 0], 'U': [1, 1]}
Grid cell (col, row): 2 2 {'R': [2, 3], 'D': [3, 2], 'L': [2, 1], 'U': [1, 2]}
Grid cell (col, row): 2 3 {'R': [2, 3], 'D': [3, 3], 'L': [2, 2], 'U': [1, 3]}
Grid cell (col, row): 3 0 {'R': [3, 1], 'D': [3, 0],

In [3]:
# generate state coordinates in the grid word, except for the terminal state
states = []
for i in range(0, WORLD_SIZE):
    for j in range(0, WORLD_SIZE):
        # handle the top-left and bottom-right terminal state
        if (i == 0 and j == 0) or (i == WORLD_SIZE - 1 and j == WORLD_SIZE - 1):
            continue
        else:
            states.append([i, j])

print(states)

[[0, 1], [0, 2], [0, 3], [1, 0], [1, 1], [1, 2], [1, 3], [2, 0], [2, 1], [2, 2], [2, 3], [3, 0], [3, 1], [3, 2]]


In [4]:
# Figure 4.1
k = 0
figure_plot_ks = [0, 1, 2, 3, 10]
threshold_delta = 1e-9  # a very small positive number

# keep iterating until convergence
while True:
    new_world = np.zeros((WORLD_SIZE, WORLD_SIZE))
    if k in figure_plot_ks:
        print('k = {}'.format(k))
        print(world)
    k += 1
    for i, j in states:
        for action in actions:
            new_state = next_state[i][j][action]
            # expected update by the Bellman equation
            new_world[i, j] += ACTION_PROB * GAMMA * (REWARD + world[new_state[0], new_state[1]])

    # convergence check
    if np.sum(np.abs(world - new_world)) < threshold_delta:
        print('Random Policy')
        print(new_world)
        break
    world = new_world

k = 0
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
k = 1
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
k = 2
[[ 0.   -1.75 -2.   -2.  ]
 [-1.75 -2.   -2.   -2.  ]
 [-2.   -2.   -2.   -1.75]
 [-2.   -2.   -1.75  0.  ]]
k = 3
[[ 0.     -2.4375 -2.9375 -3.    ]
 [-2.4375 -2.875  -3.     -2.9375]
 [-2.9375 -3.     -2.875  -2.4375]
 [-3.     -2.9375 -2.4375  0.    ]]
k = 10
[[ 0.         -6.13796997 -8.35235596 -8.96731567]
 [-6.13796997 -7.73739624 -8.42782593 -8.35235596]
 [-8.35235596 -8.42782593 -7.73739624 -6.13796997]
 [-8.96731567 -8.35235596 -6.13796997  0.        ]]
Random Policy
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
